# Business Understanding and Set-up

## Background and Key Question

## Feature Glossary

[LINK](https://github.com/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb)

| **FEATURE** | **DESCRIPTION** |
| :----- | :----- |
| **name** | header of Airbnb listing |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |



## Dataset Glossary

| **DATASET** | **DESCRIPTION** |
| :----- | :----- |
| **data_raw** | Originally imported dataset listings.csv.gz (February 2020) |
| **data** | Naming for main working dataset throughout all notebooks |
| **data_clean** | Export from Notebook 1-Clean, import for Notebooks 2-EDA and 3-Feature Engineering |
| **data_engineered** | Export from Notebook 3-Feature Engineering, import for Notebook 4-Predictive Modeling |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |
| **** |  |



## Target Feature(s) and Metric(s)

**Target 1**:
- Feature: Occupancy class
- Metric: F1-Score

## Libraries and Dashboard

In [142]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
import os, glob
import geopandas as gpd
%matplotlib inline

In [143]:
#Dashboard
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 100)
pd.options.display.max_seq_items = 300
#pd.options.display.max_rows = 4000
sns.set(style="white")

# Data Mining

## Data Checks

The monthly data for Berlin is composed of various files that are briefly visualized here (based on Dec 2019):

- listings.csv.gz
- listings.csv
- reviews.csv.gz
- reviews.csv
- calendar.csv.gz
- neighbourhoods.csv
- neighbourhoods.geojson

**listings.csv.gz**

In [144]:
# Display contents of listings.csv.gz as well as its shape
data_2020_02_18_listings_gz = pd.read_csv("data/2020-02-18/listings.csv.gz")
print(data_2020_02_18_listings_gz.shape)
data_2020_02_18_listings_gz.head(3)

(25197, 106)


/opt/anaconda3/envs/nf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3176,https://www.airbnb.com/rooms/3176,20200218053611,2020-02-18,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,We welcome FAMILIES and cater especially for y...,"We are 5 min walk away from the tram M2, whic...",The apartment will be entirely yours. We are c...,Feel free to ask any questions prior to bookin...,"It’s a non smoking flat, which likes to be tre...",NaN,NaN,https://a0.muscache.com/im/pictures/243355/84a...,NaN,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",We love to travel ourselves a lot and prefer t...,within a few hours,100%,80%,f,https://a0.muscache.com/im/users/3718/profile_...,https://a0.muscache.com/im/users/3718/profile_...,Prenzlauer Berg,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'man...",t,t,"Berlin, Berlin, Germany",Prenzlauer Berg,Prenzlauer Berg Südwest,Pankow,Berlin,Berlin,10405,Berlin,"Berlin, Germany",DE,Germany,52.53500,13.41758,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",720.0,$90.00,$520.00,"$1,900.00",$300.00,$100.00,2,$20.00,62,1125,62,62,1125,1125,62.0,1125.0,3 weeks ago,t,0,0,0,221,2020-02-18,145,1,2009-06-20,2019-06-27,93.0,9.0,9.0,9.0,9.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.12
1,3309,https://www.airbnb.com/rooms/3309,20200218053611,2020-02-19,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,none,"My flat is in the middle of West-Berlin, direc...",The flat is a strictly non-smoking facility! A...,The public transportation is excellent: Severa...,I do have a strictly non-smoker-flat. Keep th...,I'm working as a freelancing photographer. My ...,House-Rules and Information ..............(deu...,NaN,NaN,https://a0.muscache.com/im/pictures/29054294/b...,NaN,4108,https://www.airbnb.com/users/show/4108,Jana,2008-11-07,"Berlin, Berlin, Germany",ENJOY EVERY DAY AS IF IT'S YOUR LAST!!! \r\n\r...,within a day,100%,100%,f,https:

**listings.csv**

In [145]:
# Display contents of listings.csv as well as its shape
data_2020_02_18_listings = pd.read_csv("data/2020-02-18/listings.csv")
print(data_2020_02_18_listings.shape)
data_2020_02_18_listings.head(2)

(25197, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.53500,13.41758,Entire home/apt,90,62,145,2019-06-27,1.12,1,221
1,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.49885,13.34906,Private room,28,7,27,2019-05-31,0.34,1,293


**reviews.csv.gz**

In [146]:
# Display contents of reviews.csv.gz as well as its shape
data_2020_02_18_reviews_gz = pd.read_csv("data/2020-02-18/reviews.csv.gz")
print(data_2020_02_18_reviews_gz.shape)
data_2020_02_18_reviews_gz.head(2)

(543302, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3176,4283,2009-06-20,21475,Milan,"excellent stay, i would highly recommend it. a..."
1,3176,134722,2010-11-07,263467,George,Britta's apartment in Berlin is in a great are...


**reviews.csv**

In [147]:
# Display contents of reviews.csv as well as its shape
data_2020_02_18_reviews = pd.read_csv("data/2020-02-18/reviews.csv")
print(data_2020_02_18_reviews.shape)
data_2020_02_18_reviews.head(2)

(543302, 2)


,listing_id,date
0,3176,2009-06-20
1,3176,2010-11-07


**calendar.csv.gz**

In [148]:
# Display contents of calendar.csv.gz as well as its shape
data_2020_02_18_cal = pd.read_csv("data/2020-02-18/calendar.csv.gz")
print(data_2020_02_18_cal.shape)
data_2020_02_18_cal.head(2)

(9197068, 7)


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,355227,2020-02-19,f,$60.00,$60.00,45.0,400.0
1,267951,2020-02-18,f,$60.00,$60.00,2.0,300.0


**neighbourhoods.csv**

In [149]:
# Display contents of neighbourhoods.csv as well as its shape
data_2020_02_18_neighb = pd.read_csv("data/2020-02-18/neighbourhoods.csv")
print(data_2020_02_18_neighb.shape)
data_2020_02_18_neighb.head(2)

(139, 2)


,neighbourhood_group,neighbourhood
0,Charlottenburg-Wilm.,Barstraße
1,Charlottenburg-Wilm.,Charlottenburg Nord


**neighbourhoods.geojson**

In [150]:
# Display contents of neighbourhoods.geojson as well as its shape
data_2020_02_18_neighb_geojson = gpd.read_file('data/2020-02-18/neighbourhoods.geojson')
print(data_2020_02_18_neighb_geojson.shape)
data_2020_02_18_neighb_geojson.head(2)

(140, 3)


,neighbourhood,neighbourhood_group,geometry
0,Blankenfelde/Niederschönhausen,Pankow,"MULTIPOLYGON (((13.41191 52.61487, 13.41183 52..."
1,Helmholtzplatz,Pankow,"MULTIPOLYGON (((13.41405 52.54929, 13.41422 52..."


## Data Import

**Create main dataset (listings on February 22nd, i.e. pre-COVID-19)**

In [151]:
# Import dataset as DataFrame (as csv-file)
data_raw = pd.read_csv("data/2020-02-18/listings.csv.gz")

/opt/anaconda3/envs/nf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [152]:
# Assign data_raw to data (in order to always keep a freshly imported data_raw) and set id as index
data = data_raw.copy()
data.set_index('id', inplace=True)

# Data Cleaning

## Pre-cleaning

In [153]:
# Display shape of "data"
data.shape

(25197, 105)

In [154]:
# Display head(1) of "data"
data.head(1)

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3176,https://www.airbnb.com/rooms/3176,20200218053611,2020-02-18,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,We welcome FAMILIES and cater especially for y...,"We are 5 min walk away from the tram M2, whic...",The apartment will be entirely yours. We are c...,Feel free to ask any questions prior to bookin...,"It’s a non smoking flat, which likes to be tre...",NaN,NaN,https://a0.muscache.com/im/pictures/243355/84a...,NaN,3718,https://www.airbnb.com/users/show/3718,Britta,2008-10-19,"Coledale, New South Wales, Australia",We love to travel ourselves a lot and prefer t...,within a few hours,100%,80%,f,https://a0.muscache.com/im/users/3718/profile_...,https://a0.muscache.com/im/users/3718/profile_...,Prenzlauer Berg,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'man...",t,t,"Berlin, Berlin, Germany",Prenzlauer Berg,Prenzlauer Berg Südwest,Pankow,Berlin,Berlin,10405,Berlin,"Berlin, Germany",DE,Germany,52.535,13.41758,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",720.0,$90.00,$520.00,"$1,900.00",$300.00,$100.00,2,$20.00,62,1125,62,62,1125,1125,62.0,1125.0,3 weeks ago,t,0,0,0,221,2020-02-18,145,1,2009-06-20,2019-06-27,93.0,9.0,9.0,9.0,9.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,1.12


In [155]:
# Display columns of "data"
#data.columns

In [156]:
# Define columns for pre-cleaning drop
drop_columns = ['access', 'availability_30',
       'availability_60', 'availability_90',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'calendar_last_scraped',
       'calendar_updated', 'city', 
       'country', 'country_code', 
       'first_review', 
       'host_about', 'host_id',
       'host_name', 'host_neighbourhood', 'host_picture_url',
       'host_thumbnail_url', 'host_total_listings_count', 'host_since', 'host_url',
       'host_verifications',
       'interaction',
       'jurisdiction_names', 'last_review', 'last_scraped',
       'license', 'listing_url', 'market',
       'maximum_maximum_nights', 'maximum_minimum_nights',
       'maximum_nights_avg_ntm', 'medium_url', 'minimum_maximum_nights',
       'minimum_minimum_nights', 'minimum_nights_avg_ntm',
       'neighborhood_overview',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'notes',
       'picture_url', 'review_scores_accuracy',
       'review_scores_checkin', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location', 
       'review_scores_value', 'scrape_id',
       'smart_location', 'state', 'street', 
       'thumbnail_url', 'transit', 'xl_picture_url']

In [157]:
# Drop innecessary columns
data.drop(labels=drop_columns, inplace=True, axis=1)

## Inspection

In [158]:
# Display shape of "data"
data.shape

(25197, 50)

In [159]:
# Display head(5) of remaining "data"
data.head(5)

,name,summary,space,description,experiences_offered,house_rules,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3176,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,"It’s a non smoking flat, which likes to be tre...","Coledale, New South Wales, Australia",within a few hours,100%,80%,f,1.0,t,t,Prenzlauer Berg,10405,52.53500,13.41758,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",720.0,$90.00,$520.00,"$1,900.00",$300.00,$100.00,2,$20.00,62,1125,t,221,145,1,93.0,t,f,f,strict_14_with_grace_period,f,f,1,1.12
3309,BerlinSpot Schöneberg near KaDeWe,First of all: I prefer short-notice bookings. ...,"Your room is really big and has 26 sqm, is ver...",First of all: I prefer short-notice bookings. ...,none,House-Rules and Information ..............(deu...,"Berlin, Berlin, Germany",within a day,100%,100%,f,1.0,t,f,Schöneberg,10777,52.49885,13.34906,t,Apartment,Private room,1,1.0,1.0,1.0,Pull-out Sofa,"{Internet,Wifi,""Pets live on this property"",Ca...",0.0,$28.00,$175.00,$599.00,$250.00,$30.00,1,$18.00,7,35,t,293,27,1,89.0,t,f,f,strict_14_with_grace_period,f,f,1,0.34
6883,Stylish East Side Loft in Center with AC & 2 b...,NaN,Stay in a stylish loft on the second floor and...,Stay in a stylish loft on the second floor and...,none,No Pets. No loud Parties. Smoking only on th...,"Berlin, Berlin, Germany",within an hour,100%,100%,f,1.0,t,t,Friedrichshain,10243,52.51171,13.45477,t,Loft,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$125.00,$599.00,"$1,399.00",$0.00,$39.00,1,$0.00,3,90,t,8,132,8,99.0,t,f,f,moderate,f,t,1,1.08
7071,BrightRoom with sunny greenview!,Cozy and large room in the beautiful district ...,"The BrightRoom is an approx. 20 sqm (215ft²), ...",Cozy and large room in the beautiful district ...,none,Please take good care of everything during you...,"Berlin, Berlin, Germany",within an hour,100%,96%,t,2.0,t,t,Prenzlauer Berg,10437,52.54316,13.41509,t,Apartment,Private room,2,1.0,1.0,2.0,Real Bed,"{Wifi,Heating,""Family/kid friendly"",Essentials...",NaN,$33.00,NaN,NaN,$0.00,$0.00,1,$27.00,1,10,t,44,287,74,97.0,t,f,f,moderate,f,f,2,2.24
9991,Geourgeous flat - outstanding views,4 bedroom with very large windows and outstand...,"THE APPARTMENT - 4 bedroom (US, Germany: 5 roo...",4 bedroom with very large windows and outstand...,none,NaN,"Berlin, Berlin, Germany",a few days or more,0%,29%,f,1.0,t,t,Prenzlauer Berg,10405,52.53303,13.41605,f,Apartment,Entire home/apt,7,2.5,4.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",NaN,$180.00,$650.00,NaN,$400.00,$80.00,5,$10.00,6,14,t,27,8,2,100.0,t,f,f,strict_14_with_grace_period,f,f,1,0.15


In [160]:
# Describe data (summary)
data.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
host_listings_count,25148.0,3.64,37.02,0.00,1.00,1.00,2.00,1397.00
latitude,25197.0,52.51,0.03,52.34,52.49,52.51,52.53,52.66
longitude,25197.0,13.41,0.06,13.10,13.37,13.42,13.44,13.76
accommodates,25197.0,2.69,1.59,1.00,2.00,2.00,3.00,16.00
bathrooms,25177.0,1.10,0.35,0.00,1.00,1.00,1.00,8.50
bedrooms,25165.0,1.16,0.67,0.00,1.00,1.00,1.00,12.00
beds,25028.0,1.60,1.23,0.00,1.00,1.00,2.00,24.00
square_feet,412.0,499.73,517.47,0.00,0.00,431.00,753.00,4639.00
guests_included,25197.0,1.36,0.90,1.00,1.00,1.00,1.00,16.00
minimum_nights,25197.0,7.11,26.97,1.00,2.00,3.00,4.00,1124.00


In [161]:
# List datatypes (data.info()) (pre-cleaning)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25197 entries, 3176 to 42361801
Data columns (total 50 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   name                              25146 non-null  object 
 1   summary                           23913 non-null  object 
 2   space                             16322 non-null  object 
 3   description                       24677 non-null  object 
 4   experiences_offered               25197 non-null  object 
 5   house_rules                       12717 non-null  object 
 6   host_location                     25065 non-null  object 
 7   host_response_time                13457 non-null  object 
 8   host_response_rate                13457 non-null  object 
 9   host_acceptance_rate              16826 non-null  object 
 10  host_is_superhost                 25148 non-null  object 
 11  host_listings_count               25148 non-null  float64
 12

In [162]:
# Show maximum/minimum value for each numerical column
#num_features = list(data.columns[data.dtypes!=object])
#data[num_features].max()
#data[num_features].min()

Several rows with unusually high values can be identified and may in some cases be dropped at a certain threshold during data handling. Some particular features include:

| **FEATURE** | **MAX_VALUE** |
| :----- | :----- |
| **calculated_host_listings_count** | 55 |
| **accommodates** | 16 |
| **bedrooms** | 12 |
| **beds** | 24 |
| **minimum_nights** | 1.124 |
| **maximum_nights** | 10.000 |
| **number_of_reviews_ltm** | 516 (potentially misleading; actually had less reviews on Airbnb |
| **price** | 8.983 |

In [163]:
# List unique entries per column
data.nunique()

name                                24468
summary                             23108
space                               15650
description                         24205
experiences_offered                     1
house_rules                         11299
host_location                        1094
host_response_time                      4
host_response_rate                     65
host_acceptance_rate                   99
host_is_superhost                       2
host_listings_count                    48
host_has_profile_pic                    2
host_identity_verified                  2
neighbourhood                          92
zipcode                               397
latitude                            10311
longitude                           13849
is_location_exact                       2
property_type                          36
room_type                               4
accommodates                           16
bathrooms                              17
bedrooms                          

In [164]:
# List missing values (pre-cleaning)

def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)
    
count_missing(data)

square_feet               24785
monthly_price             22993
weekly_price              22134
house_rules               12480
host_response_rate        11740
host_response_time        11740
security_deposit           9660
space                      8875
host_acceptance_rate       8371
cleaning_fee               7374
review_scores_rating       4959
reviews_per_month          4449
summary                    1284
zipcode                     523
description                 520
beds                        169
host_location               132
name                         51
host_is_superhost            49
host_listings_count          49
host_identity_verified       49
host_has_profile_pic         49
bedrooms                     32
bathrooms                    20
dtype: int64


## Observations

- **host_response_rate** and **host_response_time** are unfortunately not available for half of the dataset and consequently the columns have been removed
- **review_scores** are difficult to replace if they do not exist, but at 0 they will distort the modeling. Hence, missing values are set to mean of the column
- listings without **name** and the few rows without enhanced **host information** (e.g. superhost), **bedrooms** or **bathrooms** are removed and not substantial in number
- missing values for **summary** and **description** are replaced with "" and kept in order to calculate length during feature engineering
- several features with missing values will be directly converted to 1/0 for simplification (**house_rules, security_deposit, space, cleaning_fee, monthly_price, weekly_price**)


## Data Handling

**Handle wrong/varying datatypes**

In [165]:
# Convert numeric objects to float
data.price = [float(i.strip("$").replace(",","")) for i in data.price]
data.extra_people = [float(i.strip("$").replace(",","")) for i in data.extra_people]
data.zipcode = ["zip_"+str(i)[:5] for i in data.zipcode]

**Handle missing/incorrect values**

In [166]:
# Convert columns with missing values to 1/0
data.house_rules.where(data.house_rules.isnull(), 1, inplace=True)
data.house_rules.fillna(0, inplace=True)

data.security_deposit.where(data.security_deposit.isnull(), 1, inplace=True)
data.security_deposit.fillna(0, inplace=True)

data.space.where(data.space.isnull(), 1, inplace=True)
data.space.fillna(0, inplace=True)

data.cleaning_fee.where(data.cleaning_fee.isnull(), 1, inplace=True)
data.cleaning_fee.fillna(0, inplace=True)

data.monthly_price.where(data.monthly_price.isnull(), 1, inplace=True)
data.monthly_price.fillna(0, inplace=True)

data.weekly_price.where(data.weekly_price.isnull(), 1, inplace=True)
data.weekly_price.fillna(0, inplace=True)

In [167]:
# Fill missing values of "beds" with 0 and then set all with "bed_type" Real Bed to at least 1
data.beds.fillna(0, inplace=True)
data.beds = np.where((data.beds==0) & (data.bed_type=="Real Bed"), 1, data.beds)

In [168]:
# Set all with "bathrooms" 0 to at least 0.5
data.bathrooms = np.where(data.bathrooms==0, 0.5, data.bathrooms)

In [169]:
# Fill review_scores with mean
data.review_scores_rating.fillna(data.review_scores_rating.mean(), inplace=True)
#data.review_scores_value.fillna(data.review_scores_value.mean(), inplace=True)
#data.review_scores_checkin.fillna(data.review_scores_checkin.mean(), inplace=True)
#data.review_scores_location.fillna(data.review_scores_location.mean(), inplace=True)
#data.review_scores_communication.fillna(data.review_scores_communication.mean(), inplace=True)
#data.review_scores_accuracy.fillna(data.review_scores_accuracy.mean(), inplace=True)
#data.review_scores_cleanliness.fillna(data.review_scores_cleanliness.mean(), inplace=True)

In [170]:
# Fill missing text values with ""
data.summary.fillna("", inplace=True)
data.description.fillna("", inplace=True)

**Add select amenities as column to data**

In [171]:
# Create temporary list with all amenities per listing
amenities_temp = [data.amenities[i].strip("{").strip("}").split(',') for i in data.index]

In [172]:
# Add all amenities to single list in order to count occurrences
amenities = []
for lst in amenities_temp:
    for item in lst:
        amenities.append(item)
amenities = pd.Series(amenities)

In [173]:
# Display count of individual amenities
#amenities.value_counts()

Out of the full list of amenities, not all will have a significant impact on the price. For the purpose of this analysis, an initial selection has been made and then enhanced by some great [previous work](https://github.com/L-Lewis/Airbnb-neural-network-price-prediction/blob/master/Airbnb-price-prediction.ipynb) on selecting relevant amenities. Additionally, most amenities with a split of more than 90/10 between 1/0 have been **removed (strikethrough in the list)** - except for some that were deemed substantial (24-hour check-in, breakfast, essentials, nature and views)

| **NEW COLUMN** | **PREVIOUS AMENITY/IES** |
| :----- | :----- |
| <s>**am_check_in_24h**</s> | <s>24-hour check-in</s> |
| **<s>am_air_con</s>** | <s>Air conditioning/central air conditioning</s> |
| **am_balcony** | Balcony/patio or balcony |
| **am_nature_and_views** | Beach view/beachfront/lake access/mountain view/ski-in ski-out/waterfront (i.e. great location/views) |
| **am_breakfast** | Breakfast |
| **am_tv** | Cable TV/TV |
| **am_coffee_machine** | Coffee maker/espresso machine |
| **am_cooking_basics** | Cooking basics |
| **am_white_goods** | Dishwasher/Dryer/Washer/Washer and dryer |
| **am_elevator** | Elevator |
| <s>**am_gym**</s> | <s>Exercise equipment/gym/private gym/shared gym</s> |
| **am_essentials** | Essentials |
| **am_child_friendly** | Family/kid friendly, or anything containing 'children' |
| **am_parking** | Free parking on premises/free street parking/outdoor parking/paid parking off premises/paid parking on premises |
| <s>**am_outdoor_space**</s> | <s>Garden or backyard/outdoor seating/sun loungers/terrace</s> |
| <s>**am_wellness**</s> | <s>Hot tub/jetted tub/private hot tub/sauna/shared hot tub/pool/private pool/shared pool</s> |
| <s>**am_internet**</s> | <s>Internet/pocket wifi/wifi</s> |
| **am_pets_allowed** | Pets allowed/cat(s)/dog(s)/pets live on this property/other pet(s) |
| **am_private_entrance** | Private entrance |
| <s>**am_secure**</s> | <s>Safe/security system</s> |
| <s>**am_self_check_in**</s> | <s>Self check-in</s> |
| **am_smoking_allowed** | Smoking allowed |

In [174]:
# Add select amenities as distinct columns to data

#data.loc[data.amenities.str.contains('24-hour check-in'), 'am_check_in_24h'] = 1
#data.am_check_in_24h.fillna(0, inplace=True)

#data.loc[data.amenities.str.contains('Air conditioning|Central air conditioning'), 'am_air_con'] = 1
#data.am_air_con.fillna(0, inplace=True)

data.loc[data.amenities.str.contains('Balcony|Patio'), 'am_balcony'] = 1
data.am_balcony.fillna(0, inplace=True)
#print(data.am_balcony.value_counts())

data.loc[data.amenities.str.contains('Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'), 'am_nature_and_views'] = 1
data.am_nature_and_views.fillna(0, inplace=True)
#print(data.am_nature_and_views.value_counts())

data.loc[data.amenities.str.contains('Breakfast'), 'am_breakfast'] = 1
data.am_breakfast.fillna(0, inplace=True)
#print(data.am_breakfast.value_counts())

data.loc[data.amenities.str.contains('TV'), 'am_tv'] = 1
data.am_tv.fillna(0, inplace=True)
#print(data.am_tv.value_counts())

data.loc[data.amenities.str.contains('Coffee maker|Espresso machine'), 'am_coffee_machine'] = 1
data.am_coffee_machine.fillna(0, inplace=True)
#print(data.am_coffee_machine.value_counts())

data.loc[data.amenities.str.contains('Cooking basics'), 'am_cooking_basics'] = 1
data.am_cooking_basics.fillna(0, inplace=True)
#print(data.am_cooking_basics.value_counts())

data.loc[data.amenities.str.contains('Dishwasher|Dryer|Washer'), 'am_white_goods'] = 1
data.am_white_goods.fillna(0, inplace=True)
#print(data.am_white_goods.value_counts())

data.loc[data.amenities.str.contains('Elevator'), 'am_elevator'] = 1
data.am_elevator.fillna(0, inplace=True)
#print(data.am_elevator.value_counts())

data.loc[data.amenities.str.contains('Essentials'), 'am_essentials'] = 1
data.am_essentials.fillna(0, inplace=True)
#print(data.am_essentials.value_counts())

#data.loc[data.amenities.str.contains('Exercise equipment|Gym|gym'), 'am_gym'] = 1
#data.am_gym.fillna(0, inplace=True)

data.loc[data.amenities.str.contains('Family/kid friendly|Children|children'), 'am_child_friendly'] = 1
data.am_child_friendly.fillna(0, inplace=True)
#print(data.am_child_friendly.value_counts())

data.loc[data.amenities.str.contains('parking'), 'am_parking'] = 1
data.am_parking.fillna(0, inplace=True)
#print(data.am_parking.value_counts())

#data.loc[data.amenities.str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'am_outdoor_space'] = 1
#data.am_outdoor_space.fillna(0, inplace=True)

#data.loc[data.amenities.str.contains('Hot tub|Jetted tub|hot tub|Sauna|Pool|pool'), 'am_wellness'] = 1
#data.am_wellness.fillna(0, inplace=True)

#data.loc[data.amenities.str.contains('Internet|Pocket wifi|Wifi'), 'am_internet'] = 1
#data.am_internet.fillna(0, inplace=True)

data.loc[data.amenities.str.contains('Pets|pet|Cat(s)|Dog(s)'), 'am_pets_allowed'] = 1
data.am_pets_allowed.fillna(0, inplace=True)
#print(data.am_pets_allowed.value_counts())

data.loc[data.amenities.str.contains('Private entrance'), 'am_private_entrance'] = 1
data.am_private_entrance.fillna(0, inplace=True)
#print(data.am_private_entrance.value_counts())

#data.loc[data.amenities.str.contains('Safe|Security system'), 'am_secure'] = 1
#data.am_secure.fillna(0, inplace=True)

#data.loc[data.amenities.str.contains('Self check-in'), 'am_self_check_in'] = 1
#data.am_self_check_in.fillna(0, inplace=True)

data.loc[data.amenities.str.contains('Smoking allowed'), 'am_smoking_allowed'] = 1
data.am_smoking_allowed.fillna(0, inplace=True)
#print(data.am_smoking_allowed.value_counts())


/opt/anaconda3/envs/nf/lib/python3.6/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


**Remove low-frequency classes from categorical columns**

In [175]:
# Change neighbourhoods that make up <0.1% of data to "other"
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<(0.001*len(data)), 'nb_other') if x.name=='neighbourhood' else x)

In [176]:
# Change zipcodes that make up <0.1% of data to "other"
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<(0.001*len(data)), 'zip_other') if x.name=='zipcode' else x)

**Drop irrelevant rows**

In [177]:
# Drop irrelevant rows with few missing values
data.dropna(subset=["name", "host_is_superhost", "bedrooms", "bathrooms", "zipcode"], inplace=True)

In [178]:
# Remove non-residential property_types (e.g. hotels, hostels, ...)
data = data[data.property_type.isin(["Apartment", "Condominium", "Loft", "House", "Townhouse", 
                                     "Guest suite", "Bed and breakfast", "Bungalow", "Villa"])]

In [179]:
# Remove "poor" listings (value above/below a certain threshold)
data = data[data.price < 500]
data = data[data.price >= 10]
data = data[data.minimum_nights <= 100]

**Final check and cleaning before export**

In [180]:
# Drop further columns
data.drop(["bed_type", "experiences_offered", "has_availability", "host_acceptance_rate", "host_location", 
           "host_response_rate", "host_response_time", "property_type", "number_of_reviews", "number_of_reviews_ltm",
           "requires_license", "is_business_travel_ready", "host_has_profile_pic", "host_listings_count",
           "require_guest_profile_picture", "require_guest_phone_verification",
           "reviews_per_month", "square_feet"], inplace=True, axis=1)

| **FEATURE(S)** | **NOTES** |
| :----- | :----- | 
| **bed_type** | over 97% of values were "Real Bed", hence little added value |
| **experiences_offered** | all values are "none" |
| **has_availability** | all values are "t" |
| **requires_license, host_has_profile_pic** | almost all values are "t" |
| **is_business_travel_ready** | all values are "f" |
| **require_guest_xxx** | almost all values are "f" |
| **host_listings_count** | calculated_host_listings_count appears to be a sanitized version (ranges from 1 to 55) of host_listings_count (has values 0 and highest is 1397) |
 **other host_xyz** | too many missing values |
| **reviews_per_month, number_of_reviews(_ltm)** | reviews in last 2 yrs are calculated in feature engineering |
| **square_feet** | too many missing values |
| **property_type** | 90% of values are "apartment", too many unique values to sensibly classify |

In [181]:
# List datatypes (data.info()) (post-cleaning)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23896 entries, 3176 to 42361801
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            23896 non-null  object 
 1   summary                         23896 non-null  object 
 2   space                           23896 non-null  int64  
 3   description                     23896 non-null  object 
 4   house_rules                     23896 non-null  int64  
 5   host_is_superhost               23896 non-null  object 
 6   host_identity_verified          23896 non-null  object 
 7   neighbourhood                   23896 non-null  object 
 8   zipcode                         23896 non-null  object 
 9   latitude                        23896 non-null  float64
 10  longitude                       23896 non-null  float64
 11  is_location_exact               23896 non-null  object 
 12  room_type                 

In [182]:
# List missing values (post-cleaning)

def count_missing(data):
    null_cols = data.columns[data.isnull().any(axis=0)]
    X_null = data[null_cols].isnull().sum()
    X_null = X_null.sort_values(ascending=False)
    print(X_null)
    
count_missing(data)

Series([], dtype: float64)


As we can see, we have got rid of all the missing values

In [183]:
# Sort columns in dataset
data = data.reindex(sorted(data.columns, reverse=False), axis=1)

In [184]:
# Display cleaned dataset
print(data.shape)
data.head(3)

(23896, 46)


,accommodates,am_balcony,am_breakfast,am_child_friendly,am_coffee_machine,am_cooking_basics,am_elevator,am_essentials,am_nature_and_views,am_parking,am_pets_allowed,am_private_entrance,am_smoking_allowed,am_tv,am_white_goods,amenities,availability_365,bathrooms,bedrooms,beds,calculated_host_listings_count,cancellation_policy,cleaning_fee,description,extra_people,guests_included,host_identity_verified,host_is_superhost,house_rules,instant_bookable,is_location_exact,latitude,longitude,maximum_nights,minimum_nights,monthly_price,name,neighbourhood,price,review_scores_rating,room_type,security_deposit,space,summary,weekly_price,zipcode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3176,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",221,1.0,1.0,2.0,1,strict_14_with_grace_period,1,This beautiful first floor apartment is situa...,20.0,2,t,f,1,f,t,52.53500,13.41758,1125,62,1,Fabulous Flat in great Location,Prenzlauer Berg,90.0,93.0,Entire home/apt,1,1,This beautiful first floor apartment is situa...,1,zip_10405
3309,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,"{Internet,Wifi,""Pets live on this property"",Ca...",293,1.0,1.0,1.0,1,strict_14_with_grace_period,1,First of all: I prefer short-notice bookings. ...,18.0,1,f,f,1,f,t,52.49885,13.34906,35,7,1,BerlinSpot Schöneberg near KaDeWe,Schöneberg,28.0,89.0,Private room,1,1,First of all: I prefer short-notice bookings. ...,1,zip_10777
6883,2,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",8,1.0,1.0,1.0,1,moderate,1,Stay in a stylish loft on the second floor and...,0.0,1,t,f,1,f,t,52.51171,13.45477,90,3,1,Stylish East Side Loft in Center with AC & 2 b...,Friedrichshain,125.0,99.0,Entire home/apt,1,1,,1,zip_10243


**Export data_clean**

In [185]:
# Export dataset for further use in 2_Airbnb_EDA and 3_Airbnb_Feature_Engineering
data.to_pickle("saves/data_clean.pkl")

In [186]:
# Alternative: Export with to_csv and save dtypes separately
#data.to_csv(r'saves/data_clean.csv', index = True)
#data.dtypes.to_frame('types').to_csv('saves/types_clean.csv')

**BACKUP**

In [187]:
# Import Airbnb listing data for the time period 04/2018-03/2020 (2 years)
#all_files = glob.glob(os.path.join("data", "*.csv.gz"))
#all_df = []
#for f in all_files:
#    df = pd.read_csv(f, sep=',')
#    df['file'] = f.split('/')[-1]
#    all_df.append(df)
#data_raw = pd.concat(all_df, ignore_index=True, sort=True)